In [6]:
import pandas as pd
import re

# 1. Carregar os dados
df = pd.read_csv('C:/Users/Allyson/Documents/Projeto CD/base de dados/mortalidade_amazonas_2023.csv', sep=';', encoding='utf-8', dtype=str)

# 2. Exibir estrutura básica
print("Dimensões da base:", df.shape)
print("\nColunas disponíveis:", df.columns.tolist())

# 3. Converter datas
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')

# 4. Identificar e contar valores faltantes
missing_counts = df.isna().sum()
print("\nValores faltantes por coluna:\n", missing_counts[missing_counts > 0])

# 5. Remover registros com:
# - data de óbito inválida
# - causa básica ausente
df = df.dropna(subset=['DTOBITO', 'CAUSABAS'])

# 6. Tratar e filtrar apenas doenças respiratórias (CID-10 J00 a J99)
def is_cid_respiratoria(cid):
    if pd.isna(cid):
        return False
    cid_clean = cid.strip().upper()
    match = re.match(r'^[*]?J(\d{2})', cid_clean)
    return match and 0 <= int(match.group(1)) <= 99

df['doenca_respiratoria'] = df['CAUSABAS'].apply(is_cid_respiratoria)

# 7. Converter idade para número (se for usar depois em visualizações)
df['IDADE'] = pd.to_numeric(df['IDADE'], errors='coerce')

# 8. Criar colunas auxiliares (ano, mês)
df['ano'] = df['DTOBITO'].dt.year
df['mes'] = df['DTOBITO'].dt.month

# 9. Visualizar o resultado do tratamento
print("\nDados tratados (primeiras 5 linhas):")
print(df.head())

# 10. Resumo da quantidade de óbitos respiratórios
print("\nTotal de óbitos respiratórios identificados:", df['doenca_respiratoria'].sum())


Dimensões da base: (1465610, 86)

Colunas disponíveis: ['contador', 'ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL', 'CODMUNNATU', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB', 'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE', 'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'NUMEROLOTE', 'DTINVESTIG', 'DTCADASTRO', 'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'OPOR_DO', 'CAUSAMAT', 'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA', 'NUDIASOBCO', 'DTCADINV', 'TPOBITOCOR', 'DTCONINV', 'FONTES', 'TPRESGINFO', 'TPNIVELINV', 'DT

In [13]:
# 2. Converter data de óbito
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'], format='%d%m%Y', errors='coerce')

# 3. Função para filtrar CID-10 respiratórios (J00 a J99)
def is_respiratoria(cid):
    if pd.isna(cid):
        return False
    cid = cid.strip().upper()
    match = re.match(r'^[*]?J(\d{2})', cid)
    return bool(match) and 0 <= int(match.group(1)) <= 99

# 4. Filtrar registros respiratórios e criar df_respiratorias
df['doenca_respiratoria'] = df['CAUSABAS'].apply(is_respiratoria)
df_respiratorias = df[df['doenca_respiratoria'] == True].copy()

# 5. Limpeza e tratamento dos dados em df_respiratorias
# Datas
df_respiratorias['DTNASC'] = pd.to_datetime(df_respiratorias['DTNASC'], format='%d%m%Y', errors='coerce')

# Numéricos
colunas_numericas = ['IDADE', 'PESO', 'QTDFILVIVO', 'QTDFILMORT', 'IDADEMAE']
for col in colunas_numericas:
    if col in df_respiratorias.columns:
        df_respiratorias[col] = pd.to_numeric(df_respiratorias[col], errors='coerce')

# SEXO (1: masculino, 2: feminino)
df_respiratorias['SEXO'] = df_respiratorias['SEXO'].map({'1': 'Masculino', '2': 'Feminino'}).fillna('Ignorado')

# RACACOR (traduzido)
if 'RACACOR' in df_respiratorias.columns:
    cod_raca = {
        '1': 'Branca',
        '2': 'Preta',
        '3': 'Amarela',
        '4': 'Parda',
        '5': 'Indígena',
        '9': 'Ignorado'
    }
    df_respiratorias['RACACOR'] = df_respiratorias['RACACOR'].map(cod_raca).fillna('Ignorado')

# CAUSABAS e CAUSABAS_O: remove asteriscos e espaços
for col in ['CAUSABAS', 'CAUSABAS_O']:
    if col in df_respiratorias.columns:
        df_respiratorias[col] = df_respiratorias[col].str.strip().str.replace('*', '', regex=False)

# Colunas auxiliares: ano e mês
df_respiratorias['ano'] = df_respiratorias['DTOBITO'].dt.year
df_respiratorias['mes'] = df_respiratorias['DTOBITO'].dt.month


print("\n🔍 Colunas com valores faltantes:")
print(df_respiratorias.isna().sum()[df_respiratorias.isna().sum() > 0])

# salvar a versão final
df_respiratorias.to_csv('C:/Users/Allyson/ENG4040/base_dados/mortalidade_respiratorias_2023_tratado.csv', sep=';', index=False, encoding='utf-8')



🔍 Colunas com valores faltantes:
HORAOBITO       2389
NATURAL         8022
CODMUNNATU     10491
DTNASC           133
ESTCIV          6385
               ...  
ALTCAUSA      167957
CAUSABAS_O        46
TPPOS          64733
TP_ALTERA     169279
CB_ALT        169271
Length: 70, dtype: int64
